# Introduction to basic functionalities of HOOMD-Organics

## Overview:

This tutorial introduces some basic functionalities of the HOOMD-Organics package including:

- Initializing molecule structures
- Assembling molecules in a box
- Applying force fields
- Running MD simulations using different methods




## Step-by-step guide for running MD simulation for a box of  Polyphenylene sulfide (PPS) polymers:
In this tutorial, we will run a molecular dynamics simulation of Polyphenylene sulfide (PPS) polymers using the HOOMD-Organics package.
HOOMD-Organics uses the [`HOOMD-blue`](https://hoomd-blue.readthedocs.io/en/v4.1.0/) simulation engine in the background to run different methods of simulation, and utilizes many functionalities from [`mBuild`](https://mbuild.mosdef.org/en/stable/) and [GMSO](https://gmso.mosdef.org/en/stable/) to initialize molecular structures, apply forcefields and prepare the information necessary to run a simulation.

In summary, the `HOOMD-Organics` package has three main classes:

-  `Molecule`: This class is used to define the structure of a molecule (for example the structure of a polymer built from a monomer). This class allows us to implement recipies for building complex structures.

- `System`: This class is used to assemble molecules into a box and to capture the initial `gsd` snapshot of the system. It also applies the forcefiled to the system and prepares the required forces for the simulation.

- `Simulation`: This class is used to run the simulation using the `HOOMD-blue` simulation engine. In order to initialize a simulation, a `gsd` snapshot of the system and a list of `Hoomd` forces are required.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import hoomd_organics

### Step 1: Initializing the Molecule
In this example, we are using the pre-defined recipe for building PPS molecules defined in `HOOMD_Organics`'s library. The `PPS` class is a subclass of the `Molecule` class. This class includes all the necessary information for building the PPS chains, including the monomer structure and how the monomers bond to create a chain. All we need to specify is the polymer chain length and how many polymer chains we want to create. In this example, we will create a system of 30 PPS chains with a length of 8.

In [ ]:
from hoomd_organics.library import PPS

molecules = PPS(num_mols=30, lengths=8)

### Step 2: Initializing the System
In this step, we will use the `Pack` class, which is a subclass of the `System` class, to pack a box of PPS molecules in a random fashion at a given density (density unit is `g/cm^3`). The `System` class creates the box, organizes molecules within the box, applies the forcefield (if provided) to the system and creates
the initial state of the system in form of a HOOMD snapshot. 
When a forcefield object (obtained from XML-based forcefields) is provided to the `System` class, the `System` automatically generates the list of HOOMD force objects defining bonded and non-bonded interactions by applying the forcefield to the system. 

Alternatively, users can initiate their own custom list of HOOMD force objects. In such cases, they should skip passing the `force_field` parameter during system setup and instead provide it directly to the `Simulation` class in the subsequent step. This approach allows for a greater flexibility in customizing the force interactions within the simulation, especially in cases where XML-based forcefields are not available. For more examples, see [the coarse graining tutorial](3-coarse-graining.ipynb).

In this example, the `Pack` class invokes mBuild's `fill_box` method in the background, which efficiently places molecules within a box in a randomized manner without overlaps. This method uses [PACKMOL](https://m3g.github.io/packmol/) to fill the box.
We use the pre-defined `OPLS` forcefield class, which was created from the [OPLS](https://en.wikipedia.org/wiki/OPLS) XML forcefield, to parameterize particle interactions.

The `HOOMD-Organics` library offers some commonly used forcefields that can be employed to parameterize the interactions within specific systems. Please refer to `hoomd_organics.library.forcefields` for more examples.

We also specify the `r_cut` parameter, which is the cutoff distance for the non-bonded interactions. If `auto_scale` is set to `True`, all the parameters defined in forces will be scaled. For example, all the `epsilon` values of Leonard-Jones potentials are scaled based on the maximum `epsilon` value. Also, `scale_charges=True` will make the system charge neutral.

**A note on `density` and `packing_expand_factor` parameters:**

Given a density in `g/cm^3`, the system class calculates a box length that corresponds to the specified density. We refer to calculated length as `target_box`. The `packing_expand_factor` multiplies this calulated box length by a factor and initializes the system based on the expanded box length. The reason for this expansion is that sometimes `PACKMOL` might fail to arrange molecules if the box size is too small. To address this, we suggest initially using an expand factor (default is 5) to initiate the system. Afterward, once the simulation object is created, we can shrink the simulation box to the desired `target_box` dimension.

In [ ]:
from hoomd_organics.base import Pack
from hoomd_organics.library import OPLS_AA_PPS

system = Pack(molecules=molecules, force_field=OPLS_AA_PPS(), density=0.5, r_cut=2.5, auto_scale=True, scale_charges=True, packing_expand_factor=5)

In [ ]:
system.system.visualize()

The initial snapshot can be acquired from the `system.hoomd_snapshot` object.


In [ ]:
hoomd_snapshot = system.hoomd_snapshot

The list of HOOMD force objects applied to the system can also be obtained by accessing the `system.hoomd_forcefield` attribute. These forces correspond to the bonded and non-bonded interactions parameterized from the `OPLS` forcefield.

In [ ]:
hoomd_forces = system.hoomd_forcefield
hoomd_forces

Now, let's examine the parameters of the LJ pair force. As you can see, the values of `epsilon` have been scaled to fall within the range of  0 to 1. The scaling factor for the `epsilon` parameter, which is expressed in units of energy, can be retrieved  from `system.reference_energy`. We will discuss reference values in detail in the next tutorials.

In [ ]:
lj_force = hoomd_forces[3]

dict(lj_force.params)

### Step 3: Running the Simulation

Using the snapshot and force objects provided by the `System` class in the previous step, we can proceed to initialize the simulation. The `Simulation` class, a subclass of `hoomd.Simulation`, offers additional features and functionalities that automate simulation methods, such as updating box volume, welding process and tensile tests.

### Logging

In addition, the Simulation class provides the functionality to log snapshots of the simulation as a `gsd` trajectory file while the simulation is running. The frequency of saving these snapshots into the gsd file is controlled by the `gsd_write_freq` parameter. Furthermore, the path and name of the trajectory file is specified using the `gsd_file_name` parameter, with the default being "trajectory.gsd".

The simulation objects automatically log various simulation data, including timestep, potential energy, kinetic temperature, pressure, and volume. These data are saved in a text file, and you can specify the name of this file using the `log_file_name` parameter (the default is sim_data.txt). The frequency at which this data is logged can be set using the `log_write_freq` parameter. These features allow for the efficient monitoring and analysis of simulation progress and results.

In [ ]:
from hoomd_organics.base import Simulation

sim = Simulation(initial_state=hoomd_snapshot, forcefield=hoomd_forces, gsd_write_freq=100, log_write_freq=100)

We can access simulation box length and shrink the volume to reach the desired density specified in the system. The System class calculates the target_box lengths based on the system mass and specified density.

In [ ]:
sim.run_update_volume(n_steps=1000, period=1, kT=1, tau_kt=1, final_box_lengths=system.target_box)

Let's run the simulation for 1000 time steps using the NVT ensemble at a scaled temperature of 1.0.

In [ ]:
sim.run_NVT(n_steps=1000, kT=1.0, tau_kt=0.01)

The simulation class also allows users to run the simulation under different conditions/ensembles such as NPT ensemble, NVE ensemble and Langevin dynamics. Check out `hoomd_organics/base/simulation.py` for more functionalities.


In the upcoming tutorials, we will explore a selection of features offered by the `HOOMD-Organics` package, highlighting how they can be customized to meet specific research requirements.